## SEARCHING FOR WATERBODIES IN THE CALAKMUL BIOSPHERE RESERVE

 Jugal Patel<sup>A</sup>  
 Margaret Kalacska<sup>A</sup>, Raja Sengupta<sup>A</sup>, Rafael Reyna<sup>B</sup>

 <sup>A</sup> *Department of Geography, McGill University, Montreal, Quebec*  
 <sup>B</sup> *Departamento de Conservación de la Biodiversidad, El Colegio de la Frontera Sur, Campeche, Mexico*

---

**TABLE OF CONTENTS**


1. [CONTEXT](#1.-CONTEXT)  
    1.1 [CALAKMUL BIOSPHERE RESERVE](#1.1-CALAKMUL-BIOSPHERE-RESERVE)  
    1.2 [WATERBODIES](#1.2-WATERBODIES)
2. [DATA](#2.-DATA)  
    2.1 [PLANET](#2.1-PLANET)  
    2.1.1 [BATCH DOWNLOADS](#2.1.1-BATCH-DOWNLOADS)  
    2.2 [MOSAIC](#2.2-MOSAIC)  
    2.3 [IMAGE QUALITY ISSUES](#2.3-IMAGE-QUALITY-ISSUES)  
    2.3.1 [CLOUD MASK](#2.3.1-CLOUD-MASK)      
3. [METHODS](#3.-METHODS)  
    3.1 [CLEAR MOSAIC](#3.1-CLEAR-MOSAIC)  
    3.2 [INDICES](#3.2-INDICES)  
    3.2.1 [NORMALISED DIFFERENCE VEGETATION INDEX](#3.2.1-NORMALISED-DIFFERENCE-VEGETATION-INDEX)  
    3.2.2 [SOIL ADJUSTED VEGETATION INDEX](#3.2.2-SOIL-ADJUSTED-VEGETATION-INDEX)  
    3.2.3 [NORMALISED DIFFERENCE WATER INDEX](#3.2.3-NORMALISED-DIFFERENCE-WATER-INDEX)  
    3.2.4 [NORMALISED DIFFERENCE LAKE INDEX](#3.2.4-NORMALISED-DIFFERENCE-LAKE-INDEX)  
    3.3 [FEATURE CLASSIFICATION](#3.3-FEATURE-CLASSIFICATION)      
4. [RESULTS](#4.-RESULTS)  
    4.1 [RULE-BASED CLASSIFICATION](#4.1-RULE-BASED-CLASSIFICATION)  
    4.2 [SPATIAL LOGIC FOR WATERBODIES](#4.2-SPATIAL-LOGIC-FOR-WATERBODIES)  
5. [DISCUSSION](#5.-DISCUSSION)  
    5.1 [PLANET](#5.1-PLANET)  
    5.2 [FUTURE DIRECTIONS](#5.2-FUTURE-DIRECTIONS)      
6. [CONCLUSION](#6.-CONCLUSION)     
7. [REFERENCES](#7.-REFERENCES)  

### 1. CONTEXT

#### 1.1 CALAKMUL BIOSPHERE RESERVE

The Calakmul Biosphere Reserve (CBR) is the largest protected tropical forest in Mexico with ~ 7230 km<sup>2</sup>  host to considerable floral and faunal diversity (Garcia-Gil et al., 1999; Morales and Magana 2001; Pinzon et al., 2021). Protected in 1989, the CBR stands to lose its conservation value as climate change and increasing drought stress push animals, notably groups of white-lipped and collared peccaries (*Tayassu pecari* and *Pecari tajacu*), south into more humid environments (Reyna-Hurtado et al., 2012; O'Farril et al., 2014; Pinzon et al., 2021).

![calakmul](jp_data/calakmul_resize.png)

<center>CALAKMUL BIOSPHERE RESERVE</center>  
<center>CAMPECHE, MEXICO</center>  

Peccary groups, both white-lipped and collared, are particularly well adapted to the spatial density and distribution of waterbodies. Although collared peccaries also have ecological adaptations suited for water scarcity (Reyna-Hurtado et al., 2014), both collared and white-lipped peccaries use waterbodies for drinking and wading or splashing-in to quench thirst as well as avoid ectoparasites (Reyna-Hurtado et al., 2014; 2017). In tropical forests, like the CBR, movements of both peccary species are related to waterbodies (Hernandez 2013; Garcia-Marmolejo et al., 2015). Species best associated with waterbodies are expected to move south, in search of wetter and more humid environments. As peccary are a major source of food for jaguars, and in the dry season for people, this dynamic could lead to poor conservation and human livelihoods outcomes. Understanding where and when intermittent waterbodies are available is a critical step towards simulating the movement of peccary groups in changing environmental conditions. 

#### 1.2 WATERBODIES

While no permanent waterbodies persist in the CBR, intermittent waterbodies -- known locally as 'sartenejas', 'aguadas', and 'civales' -- become apparent based on precipitation and environmental conditions. These intermittent waterbodies are critical sources of water for both local human and wildlife populations. 

Sartenejas are very small, often less than 1 metre in diameter, depressions on limestone which fill with even low amounts of precipitation. Aguadas are 3 - 10 metres in diameter, circular, shallow, and fill with precipitation as rainwater follows topographic gradients. Civales are small, ephemeral, marshy wetlands with shallow bodies of water within them. They are referred to as a type of intermittent lake by local peoples adajacent to the CBR in Guatemala (Ensley et al., 2021). In this report, civales are distinguished from aguadas, as being larger than 10 metres in diameter. 

Depending on context (i.e., socio-ecological or biological conditions) fauna in the forest reserve rely on one, two, or all three of these intermittent waterbodies. Sartenejas, aguadas, and civales each fill according to participation and hydrological conditions (i.e., fed through networks of streams and wetlands). The relative availability of water at these waterbodies is not linear; sartenejas are maintained even when aguadas run dry as small-amounts of sporadic rain is often enough to fill a sarteneja. Within the Southern zone of the CBR, tweleve aguadas were monitored using camera-traps to determine water level, as well as faunal activity from 2014 - 2017. I intend to use this time frame for this exploratory study. The changing spatial distribution of waterbodies in the region are a proxy for understanding worsening drought, climate change, and peccary movement ecology. The aim of this study is to explore remote-sensing and programmatic methods for detection of small intermittent waterbodies in the CBR and available Planet imagery. 

### 2. DATA

#### 2.1 PLANET

Planet, founded in 2010 as Cosmolgia Inc., is a publicly traded (NYSE:PL) public benefit corporation with a stated mission "to accelerate humanity to a more sustainable, secure and prosperous world by illuminating environmental and social change." Planet sells high and medium resolution imagery, spanning the entire global daily, to potentiate the characterisation of the Earth's land surface. For 2017, and for across the CBR, Planet offers no high-resolution imagery. 

PlanetScope, Planet's medium resolution imagery, is collected at a spatial resolution of 3.7 metres, and is resampled to 3 x 3 metre pixels in availed scenes. PlanetScope collected images of the CBR throughout 2017 -- suggesting a promising set of data to work with in characterising intermittent waterbodies in the CBR. These image products, PSScene scenes, can be ordered as orthorectified and with atmospheric corrections applied.

***NOTE:***  

- Further information on PlanetScope 4-band data, as well the PSScene sensor is available:   
    - 4 band (https://developers.planet.com/docs/data/psscene4band/)    
    - PlanetScope sensor (https://developers.planet.com/docs/apis/data/sensors/)  
- PSScene, includes new 8-band products (https://www.planet.com/pulse/planet-launches-next-generation-planetscope-with-eight-spectral-bands-and-quality-improvements/) 

#### 2.1.1 *BATCH DOWNLOADS*

The following script acivates and downloads all images specified by user-specified filters. Here filters include the CBR, represented as a polygon; seasons, each represented as time-ranges; and cloud filters to aid in intrepreting imagery. 

See [geogugal/planetAPI_batchdownload.ipynb](https://github.com/geogugal/calakmul-lake-classification/blob/master/planetAPI_batchdownload.ipynb) for reference to data presented in this report. 

In [ ]:
# libraries required for downloading Planet imagery 
import os # ... for operating system dependent functionality
import sys # ... for changing py runtime environment variables
import json # ... for working with json data 
import time # ... for time related functions 
import requests # ... for sending HTTP requests 
import urllib.request # ... for opening URLs 

from datetime import datetime # ... for date related functions
from requests.auth import HTTPBasicAuth # ... for authenticating HTTP requests 

***NOTE:***  

Images will be downloaded to: PATH + 'scene_folder_name'  
An API key is required to access data  
All 'PlanetScope' sensors are designated as 'PSScene' in current [PSScene4Band documentation](https://developers.planet.com/docs/data/psscene4band/)

In [ ]:
# setup

# dry months
# scene_folder_name = '201612'
# scene_folder_name = '201701'
# scene_folder_name = '201702'

# wet months
scene_folder_name = '201709'
# scene_folder_name = '201708'
# scene_folder_name = '201707'

# setup Planet
item_type = "PSScene" # PSScene refers to PlanetScope sensors
item_types = ["PSScene"]
URL = "https://api.planet.com/data/v1" 
os.environ['PL_API_KEY'] = 'xxxx' # enter api key here
PLANET_API_KEY = os.getenv('PL_API_KEY')
session = requests.Session()
session.auth = (PLANET_API_KEY, "")
res = session.get(URL)
quick_url = "{}/quick-search".format(URL) # setup the quick search endpoint url

print("downloading scenes for: " + scene_folder_name)

# setup folder structure
# data_folder = r'data/scenes/' + scene_folder_name
data_folder = r'D:/Users/jpatel27/ponds_wlp/data/scenes/' + scene_folder_name

if not os.path.exists(data_folder):
    os.makedirs(data_folder)  # create folder if it does not exist

Filters used here include a 'geometry_filter', a 'date_range_filter', and a 'cloud_filter'. 

- 'geometry_filter' - geojson of Calakmul Biosphere Reserve  
- 'date_range_filter' - from '2017-09-01T00:00:00.000Z' to '2017-09-30T00:00:00.000Z'  
- 'cloud_filter' - with less than 50 % cloud cover   

In [ ]:
# data download for ortho_analytic_4b_sr, SEPT 2017; <50% cloud cover; CBR
with open('D:/Users/jpatel27/ponds_wlp/data/calakmul.geojson', 'r') as j:
    os.chdir(data_folder)

    # read region of interest as geojson
    geojson_geometry = json.loads(j.read())

    # --------------------------------- filters ----------------------------------#
    
    # apply region of interest filter
    geometry_filter = {"type": "GeometryFilter","field_name": "geometry","config": geojson_geometry}

    date_range_filter = {"type": "DateRangeFilter","field_name": "acquired","config": {
        "gte": "2017-09-01T00:00:00.000Z","lte": "2017-09-30T00:00:00.000Z"}}
    
    # only get images with <50% cloud coverage
    cloud_cover_filter = {"type": "RangeFilter","field_name": "cloud_cover","config": {"lte": 0.5}}
    
    # combine our geo, date, and cloud filters
    combined_filter = {"type": "AndFilter","config": [geometry_filter, date_range_filter, cloud_cover_filter]}

    # setup request
    request = {"item_types": item_types,"filter": combined_filter}
    
    # --------------------------- activate / download ---------------------------#

    # Send the POST request to the API quick search endpoint
    res = session.post(quick_url, json=request)
    # Assign the response to a variable
    geojson = res.json()
    # extract image IDs only
    image_ids = [feature['id'] for feature in geojson['features']]
    print(image_ids)

    # all available assets for image id
    for i in range(len(image_ids)):
        print("current scene:" , image_ids[i])
        id0 = image_ids[i]
        id0_url = 'https://api.planet.com/data/v1/item-types/{}/items/{}/assets'.format(item_type, id0)

        # Returns JSON metadata for assets in this ID. Learn more: planet.com/docs/reference/data-api/items-assets/#asset
        result = \
            requests.get(
                id0_url,
                auth=HTTPBasicAuth(PLANET_API_KEY, '')
            )

        # List of product types available for this sensor
        result_keys = (result.json().keys())
        # print(result_keys)

        if "ortho_analytic_4b_sr" in result.json():
            # check asset status
            status = result.json()['ortho_analytic_4b_sr']['status']

            # activate asset
            # parse out useful links
            # insert product here
            links = result.json()[u"ortho_analytic_4b_sr"]["_links"]
            self_link = links["_self"]
            activation_link = links["activate"]

            # request activation of asset:
            activate_result = \
                requests.get(
                    activation_link,
                    auth=HTTPBasicAuth(PLANET_API_KEY, '')
                )
            # assign inital status to cur_activation_status
            cur_activation_status = status
            print("current status ", datetime.now(), " ", status)
            
            # if status is already active
            if status == 'active':
                # retrieve download link
                activation_status_result = \
                    requests.get(
                        self_link,
                        auth=HTTPBasicAuth(PLANET_API_KEY, '')
                    )
                download_link = activation_status_result.json()["location"]
                # print(download_link)
                urllib.request.urlretrieve(download_link, image_ids[i])
                print("download complete")
                sys.stdout.flush()

            # if asset is not yet active
            if status != 'active':
                while cur_activation_status != 'active':
                    # poll every 30 seconds
                    time.sleep(30)
                    activation_status_result = \
                        requests.get(
                            self_link,
                            auth=HTTPBasicAuth(PLANET_API_KEY, '')
                        )
                    # update cur_activation_status to current status
                    cur_activation_status = activation_status_result.json()["status"]
                    print("current status ", datetime.now(), " ", cur_activation_status)
                    # if asset is activated
                    if cur_activation_status == 'active':
                        # retrieve download link
                        download_link = activation_status_result.json()["location"]
                        urllib.request.urlretrieve(download_link, image_ids[i])
                        print("download complete: ", download_link)
                        continue
        else:
            print(image_ids[i], ' n/a')

#### 2.2 MOSAIC

For both seasons, three months were composited to create mosaics characterizing the Calakmul Biosphere Reserve. The dry season mosaics are composites of all images collected (by the PlanetScope sensor and with specified filters) of the Calakmul Biosphere Reserve from December 2016 to February 2017. The wet season mosaics are composites of all images collected in our study area from July 2017 - September 2017. These are shown below, with a red outline of the CBR and a blue bounding box of monitored aguadas. 

![mosaic_all](jp_data/small_mosaic_all.png)

To create the above mosaics, the ENVI 5.6 *Seamless Mosaic.* and *Quick Mosaic* tools were used. The seamless mosaic differentiates itself with 'edge-feathering', colour correction, and seamlines. These methods explictly consider overlap between images. 

To run mosaicking tools, each image (also referred to as 'Planet Scene(s)' or simply 'scene(s)' throughout) must be georeferenced and be the same data type. While wavelengths may vary, the number of bands across images must be consistent. In data with variable wavelength (i.e., collected with different sensors or calibration), spectral subsets must be created to ensure image compatibility. For our mosaics, all prior images contained four bands with consistent wavelengths for Blue, Green, Red and Near Infra-Red (NIR), and were formatted by Planet as GeoTIFFs (georeferenced TIFF files). 

#### 2.3  IMAGE QUALITY ISSUES

While both tools yielded similar results, there are notable similarities and differences. All of the above mosaics, for both dry and wet seasons, remain occluded by clouds. This is expected, especially in the wet season, when precipitation is highest and cloud cover remains over the region. As well, both seasons - as defined by three-month periods - do not have full cover by the PlanetScope sensor. The dry season has missing sections in the South-East most part of the CBR. Worse, the wet season has missing images over the region where our tweleve known aguadas and their water availability are monitored. 

While missing components cannot be addressed without either expanding the time range of each season, or by combining with alternative remote-sensing products, there are three relevent methods for correcting cloudy images. That is, to create cloud masks. First, existing cloud-correction products could be imported from Planet directly. Planet's 'useable data masks' (UDM2) are not available for 2017, only dating back to August 2018 for PlanetScope 4-band imagery. Second, the 'scene-order' can be set when compositing images. Considering the amount of images available for each season, this proved to be an overwhelming task to do manually. Automating this procedure also proved difficult as limited metadata is imported with imagery, despite Planet possessing critical information, like percent-cloud-cover, for each image. If such information were to be availed - linked as metadata for each image - a script could be written to set the scene order by this attribute. Relatedly, PY-ENVI, the integration between ENVI and Python remains poorly documented. Third, cloud masks could be created within ENVI by detecting, then subsequently masking, clouds across images. This procedure is described below. 

Overall, image quality concerns are problematic for downstream analysis, and requires the project be re-scaled: reducing the spatial extent we are working with, and re-envisioning our aim. That is, it is unlikely for this exploratory study to adequately consider the entire Calakmul Biosphere Reserve as of yet. Further steps to potentiate such analysis are mentioned below in discussion. 

For more information of UDM2 for PlanetScope, please see: [planet/planetscope/udm-2/](https://developers.planet.com/docs/data/udm-2/)

#### 2.3.1 *CLOUD MASK*

Two methods for creating a cloud mask were attempted across a subset image - SCENE ID: 20170922_170743_0f2d. This image has substaintial cloud cover, cloud shadow, some haze effects, and overlays where aguadas are monitored. Unfortunately, aguadas are occluded by cloud cover and cloud shadow here. Nonetheless, I use this image to begin creating a cloud mask. 

![cloud mask](jp_data/rsz__outcloud_mosaic.png)

First, an example-based classification, relying on a manual selection of cloud and show features, resulted in the below image. Here red denotes 'cloud' while blue denotes 'cloud-shadow'. While the resulting image seems promising, it betrays in two ways: 1) clouds close to one-another 'bleed' into neighbours resulting in an overclassification of cloud; and 2) many major cloud shadows are not detected. For example, a major cloud shadow in the south east portion if the image remains undetected. It may be the case that this cloud and its shadow went undetected due to their being considerable 'haze' in the area. Spectra of haze and cloud differ and would prove difficult without further training. Notably, this example-based classification was not validated beyond visual inspection.

Overall, the example based classification may prove to be a worthwhile route for cloud-masking prior to analysis, but would also require more time with training data, validation across different parts of the scene and across both the wet and dry seasons. A (spectral) rule-based classification methodology was also attempted, but yielded considerable noise. 

![example_based](jp_data/rsz_ex_based.png)

### 3. METHODS

#### 3.1 CLEAR MOSAIC

Based on data quality issues, I chose to reconsider the spatial scale of this exploratory study. Instead of thinking through solutions for cloud cover, the remainder of this report focuses on a subset of scenes which were cloud-free. These images are of the western region of the CBR and overlaps with three of tweleve monitored aguadas. Like above, these scenes are composited with the ENVI's *Seamless Mosaic* tool. Default parameters for edge-feathering and colour correction were retained as less than 10% of any given image overlapped with others (Pan et al., 2009). Seamlines were not used. This seamless composite, the primary data product for the remainder of the study, is shown below. 


***True Colour Composite***             |  ***False Colour Composite***
:-------------------------:|:-------------------------:
![clear mosaic](jp_data/rsz_2roi_mosaic.png) | ![clear mosaic](jp_data/rsz_roi_mosaic_nir.png)

To aid in intrepreting, a false colour composite is provided above. By altering which bands are read as Blue, Green, and Red, falso colour composites allow users to visualise the Near Infra-Red band. Additionally, 'true' or 'natural' colour composites are difficult to spot subtlities with as they are often 'hazy'. In contrast, the above false colour composite distinguishes otherwise subtle differences by increasing the spectral seperation between bands - thereby increasing the intrepretibility of the mosaic. 

#### 3.2 INDICES

Indices capture ...  

To use indices, images must include wavelength, wavelength units, and full-width-half-mazimum metadata. This metadata is adapted within ENVI 5.6 ... 

#### 3.2.1 *NORMALISED DIFFERENCE VEGETATION INDEX*

NDVI ...   
used to characterize vegetation and is a common starting point...   
as expected, the majority of the tropical rainforest is indexed at > 0.5 NDVI ... 

#### 3.2.2 *SOIL ADJUSTED VEGETATION INDEX*

SAVI / OSAVI ... WAS USED TO MASK images ... 

this mask is rationalised as ... 
wherever soil is visible, a limited canopy exists... and as such, water at ground level and its associated spectra could be captured ...  
computationally intensive tasks such as the rule-based feature extraction benefit from data reduction... 

extremely high soil exposure can be neglected as ... 


![ndvi](jp_data/rsz_ndvi_savi.png)

#### 3.2.3 *NORMALISED DIFFERENCE WATER INDEX*

NDWI ... 

#### 3.2.4 *NORMALISED DIFFERENCE LAKE INDEX*

NDLI ... 

![ndvi](jp_data/rsz_ndli_ndwi.png)

#### 3.3 FEATURE EXTRACTION

to classify pixels based on specified spectra ... 

NDWI > 7 

NDLI > 1.x 

### 4. RESULTS
---

#### 4.1 RULE-BASED CLASSIFICATION

Rule-based feature extraction is a tool ENVI provides for classifying ... 
parameters
- threshold

In [ ]:
## output  histogram of area, histogram for roundness 

#### 4.2 SPATIAL LOGIC FOR WATERBODIES

OF the 28k observed polygons which matched specified spectral characteristics, and given the lack of sufficient training data 


came up with 'these' filters for x, y, and z

In [ ]:
import folium 

# function to create dynamic maps
def map_function(ctr_lat, ctr_lon, geojson_file_name,zoom_start_lvl= 14, zoom_scroll=False, color = 'NA'):
    file = open(geojson_file_name, encoding="utf8")
    text = file.read()
    m = folium.Map(location=[ctr_lat, ctr_lon], 
                   zoom_start=zoom_start_lvl,
                   zoom_control=zoom_scroll,
                   scrollWheelZoom=zoom_scroll,
                   tiles = None
                   ) # study area scene 
    if color == 'NA':
        folium.GeoJson(text, name="Lakes Classified").add_to(m)
    else:
        style1 = {'fillColor': color,'color': color}
        folium.GeoJson(text, name="Lakes Classified",style_function=lambda x:style1).add_to(m)
    # Add custom basemaps
    basemaps['Google Terrain'].add_to(m)
    basemaps['Google Satellite Hybrid'].add_to(m)
    m.add_child(folium.LayerControl(collapsed = False))
    return m

In [ ]:
# basemap options for folium maps
basemaps = {
    'Google Maps': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=m&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Maps',
        overlay = True,
        control = True
    ),
    'Google Terrain': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=p&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Terrain',
        overlay = True,
        control = True
    ),
    'Google Satellite Hybrid': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=y&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Satellite',
        overlay = True,
        control = True
    )
}

In [ ]:
map_function(18.231862185818112, 
             -89.959829903259021, 
             r"jp_data/rb_class_lakes_filtered_fixed_lessfields.geojson")

In [ ]:
map_function(18.398696553867047, -89.951932307343966 , 
             geojson_file_name = r"jp_data/rb_class_lakes_filtered_select5.geojson", 
             zoom_start_lvl = 22, 
             zoom_scroll = True,
            color = '#fff700')

In [ ]:
map_function(18.397445920347188, -89.990709032582089,
             geojson_file_name = r"jp_data/rb_class_lakes_filtered_select5.geojson", 
             zoom_start_lvl = 22, 
             zoom_scroll = True,
            color = '#fff700')

In [ ]:
map_function(18.348591542006542,-89.914949593306204,
             geojson_file_name = r"jp_data/rb_class_lakes_filtered_select5.geojson", 
             zoom_start_lvl = 22, 
             zoom_scroll = True,
            color = '#fff700')

In [ ]:
map_function(18.185718568458661,-89.995582408685834,
             geojson_file_name = r"jp_data/rb_class_lakes_filtered_select5.geojson", 
             zoom_start_lvl = 22, 
             zoom_scroll = True,
            color = '#fff700')

In [ ]:
map_function(18.184093021137567,-89.993825061377763,
             geojson_file_name = r"jp_data/rb_class_lakes_filtered_select5.geojson", 
             zoom_start_lvl = 22, 
             zoom_scroll = True,
            color = '#fff700')

### 5. DISCUSSION
---

#### 5.1 PLANET

Planet images were underwhelming given the spatial extent of CBR, inconsistent image orientations, and the lack of cloud mask products for the time high cloud cover in the region ... 

#### 5.2 FUTURE DIRECTIONS

Apply classification model  
... at scale, with an automated scene order set operation; 
... with GEE if the above is not straightforward in ENVI 
... with differentiation for wet and dry seasons  
... is roundness appropriate given pixels are ...

Collect 2019 imagery and   
... use Planet's UDM2 cloud mask

### 6. CONCLUSION
---

this exploratory study shows promise for using planet imagery... 

### 7. REFERENCES
---

Garcia-Gil et al., 1999
Pinzon et al., 2021
Reyna-Hurtado et al., 2010
Reyna-Hurtado et al., 2012
Reyna-Hurtado et al., 2014
Reyna-Hurtado et al., 2017
